In [1]:
# !conda install lightgbm

In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
# from scipy.stats import permutation_test
from scipy.spatial.distance import euclidean,cosine
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import IsolationForest
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, explained_variance_score, mean_absolute_percentage_error
# import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [3]:
from scipy.sparse.linalg import lsqr as sparse_lsqr

In [4]:
import scipy
print(scipy.__file__)

C:\Users\Admin\anaconda3\lib\site-packages\scipy\__init__.py


In [5]:
sensor_df = pd.read_csv("SensorData_small.csv")

In [6]:
sensor_df.head()
sensor_df["Amount"]=(sensor_df["Amount"]-sensor_df["Amount"].mean())/sensor_df["Amount"].std()
sensor_df["Time"]=(sensor_df["Time"]-sensor_df["Time"].mean())/sensor_df["Time"].std()
sensor_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.909969,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.287897,0
1,-1.909969,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.355366,0
2,-1.909888,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.290639,0
3,-1.909888,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.173543,0
4,-1.909807,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.060725,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34832,1.148597,1.304083,-0.285770,-0.340846,-0.046637,0.116827,0.318859,-0.199270,0.128811,0.859327,...,-0.262652,-0.637530,-0.159614,-1.294839,0.490899,1.131015,-0.078832,-0.021608,-0.288776,0
34833,1.148758,1.194245,0.232805,0.626508,0.551191,-0.474094,-0.769367,0.009781,-0.106453,-0.318803,...,-0.182625,-0.548361,0.138009,0.546797,0.178350,0.061976,-0.029636,0.013011,-0.338510,0
34834,1.148839,-1.314945,-0.408721,2.844002,-0.291116,0.550643,0.654924,-0.309365,0.489621,0.335221,...,0.066109,0.214404,-0.083451,-0.283768,0.051178,0.259297,-0.136268,-0.159265,-0.257736,0
34835,1.148920,1.252570,1.220494,-2.148191,1.156019,1.819153,-0.236395,0.629757,-0.064413,-0.749796,...,-0.208746,-0.458681,-0.372473,-1.642775,0.964670,-0.200875,0.056027,0.071958,-0.350287,0


In [7]:
sensor_val_df = sensor_df[[f"V{x}" for x in range(1,29)] + ["Class","Time","Amount"]] 

sensor_label = sensor_df['Class']
X_train, X_test, y_train, y_test = train_test_split(sensor_val_df, sensor_label, test_size=0.20, random_state=100)

sensor_val_df_pos = X_train.loc[X_train.Class==1,:][[f"V{x}" for x in range(1,29)] + ["Time","Amount"]]

sensor_val_df_neg = X_train.loc[X_train.Class==0,:][[f"V{x}" for x in range(1,29)] + ["Time","Amount"]]

del X_train["Class"]
del X_test["Class"]

In [8]:
sensor_val_df_pos

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Time,Amount
29687,-2.019001,1.491270,0.005222,0.817253,0.973252,-0.639268,-0.974073,-3.146929,-0.003159,-0.121653,...,2.839596,-1.185443,-0.142812,-0.086103,-0.329113,0.523601,0.626283,0.152440,0.966024,-0.363815
6329,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,-2.447469,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,-1.302281,-0.362764
11880,-15.819179,8.775997,-22.804686,11.864868,-9.092361,-2.386893,-16.560368,0.948349,-6.310658,-13.088891,...,-2.350634,1.036362,1.136051,-1.043414,-0.108923,0.657437,2.136424,-1.411945,-0.257112,-0.362764
30398,-3.519030,4.140867,-3.628202,5.505672,-4.057463,-0.905945,-6.652031,2.634524,-4.679402,-6.546242,...,1.582556,0.778710,-0.135707,-0.004278,0.032706,0.362014,0.900925,0.554897,0.991967,-0.327171
6446,0.702710,2.426433,-5.234513,4.416661,-2.170806,-2.667554,-3.878088,0.911337,-0.166199,-5.009249,...,0.551180,-0.009802,0.721698,0.473246,-1.959304,0.319476,0.600485,0.129305,-1.289915,-0.362764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15781,-25.266355,14.323254,-26.823673,6.349248,-18.664251,-4.647403,-17.971212,16.633103,-3.768351,-8.303239,...,1.780701,-1.861318,-1.188167,0.156667,1.768192,-0.219916,1.411855,0.414656,0.289880,0.070616
6903,-2.535852,5.793644,-7.618463,6.395830,-0.065210,-3.136372,-3.104557,1.823233,-3.878658,-7.297803,...,0.716720,-0.448060,-0.402407,-0.288835,1.011752,0.425965,0.413140,0.308205,-1.191799,-0.362764
33276,-7.923891,-5.198360,-3.000024,4.420666,2.272194,-3.394483,-5.283435,0.131619,0.658176,-0.794994,...,-0.734308,-0.599926,-4.908301,0.410170,-1.167660,0.520508,1.937421,-1.552593,1.093881,-0.313249
15751,-24.590245,14.044567,-26.278701,6.320089,-18.224513,-4.609968,-17.681003,16.213627,-3.794093,-8.372753,...,1.777063,-1.804874,-1.140605,0.152234,1.715997,-0.220471,1.434951,0.422492,0.287294,0.070616


In [9]:
param_grid = {
      'learning_rate': [0.1, 0.01,0.001,0.0001],
      'n_estimators': [1000,2000,3000,5000],
      'max_depth': [4, 6,10,20],
      'colsample_bytree': [0.7, 0.8, 0.9],
      'subsample': [0.7, 0.8, 0.9],
      'min_child_samples': [1, 5, 10]
  }

In [10]:
estimator = lgb.LGBMClassifier(random_state = 101)

In [11]:
model = GridSearchCV(estimator=estimator, 
                       param_grid=param_grid,
                       cv=3, 
                       n_jobs=-1, 
                       scoring='f1')
model.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LGBMClassifier(random_state=101), n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'learning_rate': [0.1, 0.01, 0.001, 0.0001],
                         'max_depth': [4, 6, 10, 20],
                         'min_child_samples': [1, 5, 10],
                         'n_estimators': [1000, 2000, 3000, 5000],
                         'subsample': [0.7, 0.8, 0.9]},
             scoring='f1')

In [12]:
model_bst = model.best_estimator_

In [13]:
y_pred = model_bst.predict(X_test)

In [15]:
model_bst

LGBMClassifier(colsample_bytree=0.7, learning_rate=0.01, max_depth=4,
               min_child_samples=1, n_estimators=5000, random_state=101,
               subsample=0.7)

In [14]:
print(f"Precision : {precision_score(y_test,y_pred)}")
print(f"Recall : {recall_score(y_test,y_pred)}")
print(f"F1 Score : {f1_score(y_test,y_pred)}")
print(f"Accuracy Score : {accuracy_score(y_test,y_pred)}")

Precision : 1.0
Recall : 0.8235294117647058
F1 Score : 0.9032258064516129
Accuracy Score : 0.9995694603903559
